# Lesson 3: Agentic Search

In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [3]:
# run search
result = client.search("What is in Nvidia's new DGX GPU?",
                       include_answer=True)

# print the answer
result["answer"]


'The DGX Spark features a 1 petaflop Grace Blackwell GB10 Superchip and 128GB memory, priced at $3,999. It can run AI models with up to 200 billion parameters locally. It aims to democratize AI compute power.'

## Regular search

In [11]:
# choose location (try to change to your own city!)

city = "Mumbai"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [12]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

returning previous results due to exception reaching ddg.
https://weather.com/weather/today/l/USCA0987:1:US
https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8


In [13]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [14]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

returning previous results due to exception reaching ddg.
Website: https://weather.com/weather/today/l/USCA0987:1:US


<body><div class="appWrapper DaybreakLargeScreen LargeScreen lightTheme twcTheme DaybreakLargeScreen--appWrapper--ZkDop" id="appWrapper"><div class="region-sidebarNavigation regionSidebarNavigation"><div class="removeIfEmpty" id="WxuSidebarNavigation-sidebarNavigation-sidebar-nav-injection"><div aria-hidden="false" class="SidebarNavigation--sidebarWrapper--v+g78"><nav aria-label="Site Navigation Links" class="SidebarNavigation--sidebar--A5yHr" data-testid="sidebar-navigation"><div class="SidebarNavigation--sidebarHeader--bMAxo"><button class="SidebarNavigation--overlayControlButton--cSFvn openSidebarOverlayButton" tabindex="0" type="button"><span class="Icon--iconWrapper--vSeDL"><svg class="Icon--icon--ySD-o" fill="none" height="24" name="hamburger" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg">
<title>Hamburger</title>
<path d="M20 16C20.5523 16 21

In [15]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://weather.com/weather/today/l/USCA0987:1:US


Recents Weather Forecasts Radar & Maps News & Media Products & Account Lifestyle Specialty Forecasts Downtown, San Francisco, California Beach Hazard Statement Weather Today in Downtown, San Francisco, California 7:27 am 6:20 pm Hourly Weather - Downtown, San Francisco, California Now Cloudy 12 am Cloudy 1 am Cloudy 2 am Cloudy Don't Miss Seasonal Hub 10 Day Weather - Downtown, San Francisco, California Tonight Night Cloudy skies. Low 57F. Winds SW at 5 to 10 mph. Sat 25 Day Cloudy with occasional rain...mainly in the morning. High 66F. Winds SW at 10 to 15 mph. Chance of rain 80%. Night Mostly cloudy. Low near 55F. Winds W at 5 to 10 mph. Sun 26 Day Cloudy. High 66F. Winds SW at 10 to 20 mph. Night Generally fair. Low 52F. Winds NW at 10 to 15 mph. Mon 27 Day Some clouds in the morning will give way to mainly sunny skies for the afternoon. High 69F. Winds NNW at 5 to 10 mph. Night A mostly clear sky. Low 53F. Winds N at 5 to

## Agentic Search

In [16]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'Mumbai', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.975, 'lon': 72.826, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1761369766, 'localtime': '2025-10-25 10:52'}, 'current': {'last_updated_epoch': 1761369300, 'last_updated': '2025-10-25 10:45', 'temp_c': 31.4, 'temp_f': 88.5, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 8.3, 'wind_kph': 13.3, 'wind_degree': 124, 'wind_dir': 'SE', 'pressure_mb': 1008.0, 'pressure_in': 29.77, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 63, 'cloud': 75, 'feelslike_c': 38.8, 'feelslike_f': 101.8, 'windchill_c': 28.1, 'windchill_f': 82.6, 'heatindex_c': 31.2, 'heatindex_f': 88.2, 'dewpoint_c': 22.4, 'dewpoint_f': 72.3, 'vis_km': 4.5, 'vis_miles': 2.0, 'uv': 4.5, 'gust_mph': 11.4, 'gust_kph': 18.4}}


In [17]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "Mumbai",
        "region": "Maharashtra",
        "country": "India",
        "lat": 18.975,
        "lon": 72.826,
        "tz_id": "Asia/Kolkata",
        "localtime_epoch": 1761369766,
        "localtime": "2025-10-25 10:52"
    },
    "current": {
        "last_updated_epoch": 1761369300,
        "last_updated": "2025-10-25 10:45",
        "temp_c": 31.4,
        "temp_f": 88.5,
        "is_day": 1,
        "condition": {
            "text": "Mist",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/143.png",
            "code": 1030
        },
        "wind_mph": 8.3,
        "wind_kph": 13.3,
        "wind_degree": 124,
        "wind_dir": "SE",
        "pressure_mb": 1008.0,
        "pressure_in": 29.77,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 63,
        "cloud": 75,
        "feelslike_c": 38.8,
        "feelslike_f": 101.8,
        "windchill_c": 28.1,
        "windchill_f": 82.6,
        "heatindex_

<img src="./google_sample.png" width="800" height="600">